#**Projeto Didático de ETL com Pandas: Emissões de CO₂ por País**

##**Objetivo**

Este projeto tem como finalidade praticar e documentar o processo ETL (Extração, Transformação e Carga) utilizando exclusivamente a biblioteca pandas, com foco em dados reais de emissões de CO₂. Trata-se de um estudo didático, realizado para fins de aprendizado pessoal e desenvolvimento de portfólio na área de Ciência de Dados.

##**Fonte dos Dados**

Dataset: GCB2024v17_MtCO2_flat.csv – Zenodo


##**Referência Bibliográfica conforme exigido pelo site:**

Ritchie, H., Roser, M., & Rosado, P. (2020). CO₂ and Greenhouse Gas Emissions. Our World in Data. Disponível em https://ourworldindata.org/co2-and-greenhouse-gas-emissions


Este conjunto de dados apresenta estatísticas históricas de emissões de dióxido de carbono (CO₂) por país, desde 1750.





###Extração


In [8]:
# Instalação de biblioteca auxiliar para validação de países
!pip install pycountry

import pycountry
import pandas as pd

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# URL do dataset
url = "https://zenodo.org/records/13981696/files/GCB2024v17_MtCO2_flat.csv"

# Leitura dos dados diretamente da web
df = pd.read_csv(url)

# Verificação inicial da estrutura
df.info()
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60828 entries, 0 to 60827
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             60828 non-null  object 
 1   ISO 3166-1 alpha-3  60006 non-null  object 
 2   UN M49              60828 non-null  int64  
 3   Year                60828 non-null  int64  
 4   Total               23863 non-null  float64
 5   Coal                18412 non-null  float64
 6   Oil                 21861 non-null  float64
 7   Gas                 14660 non-null  float64
 8   Cement              24479 non-null  float64
 9   Flaring             13026 non-null  float64
 10  Other               2273 non-null   float64
 11  Per Capita          22913 non-null  float64
dtypes: float64(8), int64(2), object(2)
memory usage: 5.6+ MB


,Country,ISO 3166-1 alpha-3,UN M49,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Afghanistan,AFG,4,1750,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1,Afghanistan,AFG,4,1751,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,Afghanistan,AFG,4,1752,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,Afghanistan,AFG,4,1753,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,Afghanistan,AFG,4,1754,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


###Limpeza dos Dados

**Objetivo**: manter apenas registros úteis e válidos, removendo colunas desnecessárias, linhas nulas e países/territórios não reconhecidos oficialmente.

In [11]:
# Filtrar apenas países válidos conforme código ISO-3166 alpha-3
codigos_validos = {country.alpha_3 for country in pycountry.countries}
df = df[df['ISO 3166-1 alpha-3'].isin(codigos_validos)]

# Remoção de colunas não utilizadas
df = df.drop(columns=['Total', 'Per Capita'], errors='ignore')

# Remoção de entradas não representativas
nao_paises = {
    "Anguilla", "Antarctica", "Aruba", "Bermuda", "Bonaire, Saint Eustatius and Saba",
    "British Virgin Islands", "Christmas Island", "Cook Islands", "Curaçao", "Faeroe Islands",
    "French Polynesia", "Greenland", "Hong Kong", "Macao", "Montserrat", "New Caledonia",
    "Niue", "Pacific Islands (Palau)", "Saint Helena", "Saint Pierre and Miquelon",
    "Sint Maarten (Dutch part)", "Turks and Caicos Islands", "Wallis and Futuna Islands",
    "Kuwaiti Oil Fires", "International Shipping", "International Aviation",
    "Global", "Ryukyu Islands", "State of Palestine", "Taiwan"
}
df = df[~df["Country"].isin(nao_paises)]

# Conversão de ano para inteiro
df['Year'] = df['Year'].astype(int)

# Remoção de registros sem emissões
colunas_emissoes = ['Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other']
df = df[df[colunas_emissoes].sum(axis=1) > 0]

###Transformação

**Objetivo**: padronizar valores e criar colunas derivadas para facilitar análises posteriores.

In [12]:
# Calcular a coluna 'Total Emissions' com a soma das fontes
colunas_emissoes = ['Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other']
df.loc[:, 'Total Emissions'] = df[colunas_emissoes].sum(axis=1)

# Converter de megatoneladas para toneladas (multiplicação por 1 milhão)
colunas_emissoes = ['Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Total Emissions']
df[colunas_emissoes] = (df[colunas_emissoes] * 1_000_000).round().fillna(0).astype(int)

# Confirmar novamente que não há registros com emissões nulas
df = df[df[colunas_emissoes].sum(axis=1) > 0]

df[colunas_emissoes].head()

,Coal,Oil,Gas,Cement,Flaring,Other,Total Emissions
199,14656,0,0,0,0,0,14656
200,21068,63204,0,0,0,0,84272
201,25648,65952,0,0,0,0,91600
202,31708,59892,0,0,0,0,91600
203,37949,68307,0,0,0,0,106256


###Carga

Exportação final para arquivo Excel com múltiplas abas — uma para o dataset completo e outras específicas por país.

In [13]:
# Selecionar países para análise específica
paises = ['China']
dfs_filtrados = {pais: df[df['Country'] == pais].copy() for pais in paises}

# Adicionar dataset geral
dfs_filtrados['Geral'] = df.copy()

# Exportação para Excel com múltiplas abas
caminho_arquivo = "/content/emissoes_co2_completo_e_filtrados.xlsx"

with pd.ExcelWriter(caminho_arquivo) as writer:
    for aba, df_exportar in dfs_filtrados.items():
        df_exportar.to_excel(writer, sheet_name=aba[:31], index=False)

In [14]:
# Download (Google Colab)
from google.colab import files
files.download(caminho_arquivo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Considerações Finais

Este projeto foi desenvolvido exclusivamente para fins educacionais, com foco em aplicar conceitos fundamentais de ETL utilizando a biblioteca pandas.